# Import Libraries

In [500]:
import pandas as pd
import numpy as np
import scipy as sc
import os
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
import math
import random

# Load Dataset
- 30 is considered as test dataset
- 70 is considered as train dataset 

In [501]:
raw_account_70_new=pd.read_csv("./test_data/raw_account_70_new.csv",low_memory=False)
raw_account_30_new=pd.read_csv("./test_data/raw_account_30_new.csv",low_memory=False)
raw_data_70_new=pd.read_csv("./test_data/raw_data_70_new.csv",low_memory=False)
raw_data_30_new=pd.read_csv("./test_data/raw_data_30_new.csv",low_memory=False)
raw_enquiry_70_new=pd.read_csv("./test_data/raw_enquiry_70_new.csv",low_memory=False)
raw_enquiry_30_new=pd.read_csv("./test_data/raw_enquiry_30_new.csv",low_memory=False)

**Shape of datasets**

- raw_account_70_new:(186329, 21)
- raw_account_30_new: (79568, 21)
- raw_data_70_new: (23896, 83)
- raw_data_30_new: (10240, 83)
- raw_enquiry_70_new: (413188, 6)
- raw_enquiry_30_new: (173499, 6)

# Percentage of target


In [502]:
raw_data_70_new.Bad_label.value_counts(normalize=False)

0    22892
1     1004
Name: Bad_label, dtype: int64

In [503]:
raw_data_30_new.Bad_label.value_counts(normalize=False)

0    9778
1     462
Name: Bad_label, dtype: int64

# Feature Creation

In [504]:
raw_account_70_new['ind_acc_70']=1
raw_account_30_new['ind_acc_30']=1
raw_enquiry_70_new['ind_enq_70']=1
raw_enquiry_30_new['ind_enq_30']=1

In [505]:
#Creating unified payment history
raw_account_70_new['paymenthistory_un']=[raw_account_70_new.paymenthistory1[i].split('"')[3]+raw_account_70_new.paymenthistory2[i].split('"')[3] if raw_account_70_new.paymenthistory2[i]==raw_account_70_new.paymenthistory2[i] else raw_account_70_new.paymenthistory1[i].split('"')[3] for i in range(len(raw_account_70_new))]
raw_account_30_new['paymenthistory_un']=[raw_account_30_new.paymenthistory1[i].split('"')[3]+raw_account_30_new.paymenthistory2[i].split('"')[3] if raw_account_30_new.paymenthistory2[i]==raw_account_30_new.paymenthistory2[i] else raw_account_30_new.paymenthistory1[i].split('"')[3] for i in range(len(raw_account_30_new))] 

In [506]:
raw_account_70_new.head(10)

,dt_opened,customer_no,upload_dt,acct_type,owner_indic,opened_dt,last_paymt_dt,closed_dt,reporting_dt,high_credit_amt,...,paymenthistory2,paymt_str_dt,paymt_end_dt,creditlimit,cashlimit,rateofinterest,paymentfrequency,actualpaymentamount,ind_acc_70,paymenthistory_un
0,10-Nov-15,12265,20-Oct-15,6,1,09-Jun-13,30-Jun-14,05-Jul-14,30-Sep-15,20900.0,...,NaN,01-Sep-15,01-Jul-14,NaN,NaN,NaN,NaN,NaN,1,STDSTDSTDXXXXXXXXXXXXXXXSTDXXXXXXXXXXXXXXXSTD
1,10-Nov-15,12265,20-Oct-15,10,1,25-May-12,06-Sep-15,NaN,03-Oct-15,16201.0,...,"""""""000000000000000000000000000XXX0000000000000...",01-Oct-15,01-Nov-12,14000.0,1400.0,NaN,3.0,5603.0,1,0000000000000000000000000000000000000000000000...
2,10-Nov-15,12265,20-Oct-15,10,1,22-Mar-12,31-Aug-15,NaN,30-Sep-15,41028.0,...,"""""""0000000000000000000000000000000000000000000...",01-Sep-15,01-Oct-12,NaN,NaN,NaN,NaN,NaN,1,0000000000000000000000000000000000000000000000...
3,20-Jul-15,15606,09-Jul-15,10,1,13-Jan-06,NaN,26-Jul-07,31-Jan-09,93473.0,...,NaN,01-Jul-07,01-Feb-06,NaN,NaN,NaN,NaN,NaN,1,1200900600600600300000000000000000000000000000...
4,20-Jul-15,15606,09-Jul-15,6,1,18-Jan-15,05-May-15,NaN,31-May-15,20250.0,...,NaN,01-May-15,01-Jan-15,NaN,NaN,NaN,NaN,NaN,1,000000000000000
5,20-Jul-15,15606,09-Jul-15,6,1,14-Jan-15,02-May-15,NaN,31-May-15,20000.0,...,NaN,01-May-15,01-Jan-15,NaN,NaN,NaN,NaN,NaN,1,000000000000000
6,20-Jul-15,15606,09-Jul-15,6,1,29-Dec-14,02-May-15,NaN,31-May-15,25100.0,...,NaN,01-May-15,01-Dec-14,NaN,NaN,NaN,NaN,NaN,1,000000000000000000
7,20-Jul-15,15606,09-Jul-15,6,1,22-Oct-14,02-May-15,NaN,31-May-15,16690.0,...,NaN,01-May-15,01-Oct-14,NaN,NaN,NaN,NaN,NaN,1,000000000000000000000000
8,20-Jul-15,15606,09-Jul-15,10,1,04-Dec-12,08-May-15,NaN,31-May-15,140160.0,...,"""""""000000000000000000000000000000000000""""""",01-May-15,01-Dec-12,140000.0,NaN,39.0,NaN,6516.0,1,0000000000000000000000000000000000000000000000...
9,20-Jul-15,15606,09-Jul-15,10,1,01-Aug-12,21-May-15,NaN,26-Jun-15,NaN,...,"""""""030000000000XXX000000000000000000000000XXX0...",01-Jun-15,01-Aug-12,NaN,NaN,NaN,3.0,NaN,1,000000000XXX000000000000030XXX0000000000000000...


In [507]:
#payment_history_avg_dpd_0_29_bucket
def find_g(lst, a):
    '''Function to find index of elements in list greater than a'''
    return [i for i, x in enumerate(lst) if x >=a]
def payment_break(lst):
    condition_list=[]
    len_str_list=[]
    for i in range(len(lst)):
        if isinstance(lst[i], str)==False:
            element_append=None
            len_str=0
        else:
            str_list=list(re.findall('.{1,3}', lst[i]))
            len_str=len(str_list)
            lst_int=list(filter(lambda a: a not in ['STD','SMA','SUB','DBT','LSS','XXX'], str_list))
            lst_int=[int(i) for i in lst_int]
            if len(lst_int)!=0:
                if len(find_g(lst_int,30))>0:
                    element_append=0
                else:
                    element_append=1
            else:
                element_append=None
        condition_list.append(element_append)
        len_str_list.append(len_str)
    return condition_list,len_str_list

raw_account_70_new['payment_history_un_dpd_0_29_bucket'],raw_account_70_new['payment_history_un_len']=payment_break(lst=raw_account_70_new.paymenthistory_un)
raw_account_30_new['payment_history_un_dpd_0_29_bucket'],raw_account_30_new['payment_history_un_len']=payment_break(lst=raw_account_30_new.paymenthistory_un)
#Group by customer number in accounts dataset
raw_account_70_new_cust_count=pd.DataFrame(raw_account_70_new.groupby(['customer_no'] ,as_index=False,sort=False)[['ind_acc_70','payment_history_un_dpd_0_29_bucket','payment_history_un_len']].sum())
raw_account_70_new_cust_count['rt_pay30_sum']=raw_account_70_new_cust_count['payment_history_un_dpd_0_29_bucket']/raw_account_70_new_cust_count['ind_acc_70']
raw_account_70_new_cust_count['payment_history_mean_length']=raw_account_70_new_cust_count['payment_history_un_len']/raw_account_70_new_cust_count['ind_acc_70']

raw_account_30_new_cust_count=pd.DataFrame(raw_account_30_new.groupby(['customer_no'] ,as_index=False,sort=False)[['ind_acc_30','payment_history_un_dpd_0_29_bucket','payment_history_un_len']].sum())
raw_account_30_new_cust_count['rt_pay30_sum']=raw_account_30_new_cust_count['payment_history_un_dpd_0_29_bucket']/raw_account_30_new_cust_count['ind_acc_30']
raw_account_30_new_cust_count['payment_history_mean_length']=raw_account_30_new_cust_count['payment_history_un_len']/raw_account_30_new_cust_count['ind_acc_30']



#Merge to demographic data
raw_data_70_new=pd.merge(raw_data_70_new,raw_account_70_new_cust_count.loc[:,['customer_no','ind_acc_70','rt_pay30_sum','payment_history_mean_length']],how='left',on='customer_no')
raw_data_30_new=pd.merge(raw_data_30_new,raw_account_30_new_cust_count.loc[:,['customer_no','ind_acc_30','rt_pay30_sum','payment_history_mean_length']],how='left',on='customer_no')


In [508]:
#min_months_last_30_plus 
def find_r(lst,a):
    '''Function to find index of elements in list greater than a'''
    return [i for i, x in enumerate(lst) if x>a ]
def min_month_30(lst):
    first_30p_list=[]
    tot_30p_list=[]
    for i in range(len(lst)):
        str_list=list(re.findall('.{1,3}', lst[i]))
        str_list_upd=[int(element) if element not in ['STD','SMA','SUB','DBT','LSS','XXX'] else -999 for element in str_list]
        tot_mon_30p=len(find_r(lst=str_list_upd,a=30))
        if tot_mon_30p>0:
            num_mon_30p=max(find_r(lst=str_list_upd,a=30))+1
        else:
            num_mon_30p=-999
        if  num_mon_30p != -999:
            first_30p=len(str_list_upd)-num_mon_30p
        else:
            first_30p=9999
        first_30p_list.append(first_30p)
        tot_30p_list.append(tot_mon_30p)
    return first_30p_list,tot_30p_list
raw_account_70_new['months_last_30_plus'],raw_account_70_new['months_total_30_plus']=min_month_30(lst=raw_account_70_new.paymenthistory_un)
raw_account_30_new['months_last_30_plus'],raw_account_30_new['months_total_30_plus']=min_month_30(lst=raw_account_30_new.paymenthistory_un)
#group by getting minimum on each account
raw_account_70_min_mon=pd.DataFrame(raw_account_70_new.groupby(['customer_no'] ,as_index=False,sort=False)[['months_last_30_plus']].min())
raw_account_70_tot_mon=pd.DataFrame(raw_account_70_new.groupby(['customer_no'] ,as_index=False,sort=False)[['ind_acc_70','months_total_30_plus']].sum())
raw_account_70_tot_mon['mean_mon_30p']=raw_account_70_tot_mon['months_total_30_plus']/raw_account_70_tot_mon['ind_acc_70']

raw_account_30_min_mon=pd.DataFrame(raw_account_30_new.groupby(['customer_no'] ,as_index=False,sort=False)[['months_last_30_plus']].min())
raw_account_30_tot_mon=pd.DataFrame(raw_account_30_new.groupby(['customer_no'] ,as_index=False,sort=False)[['ind_acc_30','months_total_30_plus']].sum())
raw_account_30_tot_mon['mean_mon_30p']=raw_account_30_tot_mon['months_total_30_plus']/raw_account_30_tot_mon['ind_acc_30']

# Merge with data
raw_data_70_new=pd.merge(raw_data_70_new,raw_account_70_min_mon.loc[:,['customer_no','months_last_30_plus']],how='left',on='customer_no')
raw_data_70_new=pd.merge(raw_data_70_new,raw_account_70_tot_mon.loc[:,['customer_no','mean_mon_30p']],how='left',on='customer_no')

raw_data_30_new=pd.merge(raw_data_30_new,raw_account_30_min_mon.loc[:,['customer_no','months_last_30_plus']],how='left',on='customer_no')
raw_data_30_new=pd.merge(raw_data_30_new,raw_account_30_tot_mon.loc[:,['customer_no','mean_mon_30p']],how='left',on='customer_no')



In [509]:
#Average and maximum current age in all accounts
def current_age(lst,method):
    curr_age=[]
    for i in range(len(lst)):
        str_list=list(re.findall('.{1,3}', lst[i]))
        if method=="average":
            str_list_upd=[int(element) if element not in ['STD','SMA','SUB','DBT','LSS','XXX'] else None for element in str_list]
        else:
            str_list_upd=[int(element) if element not in ['STD','SMA','SUB','DBT','LSS','XXX'] else -999 for element in str_list]
        curr_age.append(str_list_upd[0])
    return curr_age
raw_account_70_new['current_age_max']=current_age(lst=raw_account_70_new.paymenthistory_un,method="max")
raw_account_70_new['current_age_average']=current_age(lst=raw_account_70_new.paymenthistory_un,method="average")
raw_account_30_new['current_age_max']=current_age(lst=raw_account_30_new.paymenthistory_un,method="max")
raw_account_30_new['current_age_average']=current_age(lst=raw_account_30_new.paymenthistory_un,method="average")

#group by
raw_account_70_curr_avg=pd.DataFrame(raw_account_70_new.groupby(['customer_no'] ,as_index=False,sort=False)[['current_age_average','ind_acc_70']].sum())
raw_account_70_curr_avg['current_age_average_r']=raw_account_70_curr_avg.current_age_average/raw_account_70_curr_avg.ind_acc_70
raw_account_70_curr_max=pd.DataFrame(raw_account_70_new.groupby(['customer_no'] ,as_index=False,sort=False)[['current_age_max']].max())

raw_account_30_curr_avg=pd.DataFrame(raw_account_30_new.groupby(['customer_no'] ,as_index=False,sort=False)[['current_age_average','ind_acc_30']].sum())
raw_account_30_curr_avg['current_age_average_r']=raw_account_30_curr_avg.current_age_average/raw_account_30_curr_avg.ind_acc_30
raw_account_30_curr_max=pd.DataFrame(raw_account_30_new.groupby(['customer_no'] ,as_index=False,sort=False)[['current_age_max']].max())

#merge
raw_data_70_new=pd.merge(raw_data_70_new,raw_account_70_curr_avg.loc[:,['customer_no','current_age_average_r']],how='left',on='customer_no')
raw_data_70_new=pd.merge(raw_data_70_new,raw_account_70_curr_max.loc[:,['customer_no','current_age_max']],how='left',on='customer_no')

raw_data_30_new=pd.merge(raw_data_30_new,raw_account_30_curr_avg.loc[:,['customer_no','current_age_average_r']],how='left',on='customer_no')
raw_data_30_new=pd.merge(raw_data_30_new,raw_account_30_curr_max.loc[:,['customer_no','current_age_max']],how='left',on='customer_no')



In [510]:
#total_diff_lastpaymt_opened_dt
raw_account_70_new.opened_dt=pd.to_datetime(raw_account_70_new.opened_dt)
raw_account_70_new.last_paymt_dt=pd.to_datetime(raw_account_70_new.last_paymt_dt)
raw_account_30_new.opened_dt=pd.to_datetime(raw_account_30_new.opened_dt)
raw_account_30_new.last_paymt_dt=pd.to_datetime(raw_account_30_new.last_paymt_dt)
raw_account_70_new['diff_lastpaymt_opened_dt']=raw_account_70_new.last_paymt_dt-raw_account_70_new.opened_dt
raw_account_30_new['diff_lastpaymt_opened_dt']=raw_account_30_new.last_paymt_dt-raw_account_30_new.opened_dt
raw_account_70_new['diff_lastpaymt_opened_dt']=raw_account_70_new.diff_lastpaymt_opened_dt.apply(lambda x: x / np.timedelta64(1,'D'))
raw_account_30_new['diff_lastpaymt_opened_dt']=raw_account_30_new.diff_lastpaymt_opened_dt.apply(lambda x: x / np.timedelta64(1,'D'))
#Fill Na #Cash Limit
raw_account_70_new[['cashlimit']]=raw_account_70_new[['cashlimit']].fillna(value=0)
raw_account_30_new[['cashlimit']]=raw_account_30_new[['cashlimit']].fillna(value=0)
# Fill Na. Credit Limit 
index_70=raw_account_70_new['creditlimit'].index[raw_account_70_new['creditlimit'].apply(np.isnan)]
raw_account_70_new.loc[index_70,'creditlimit']=raw_account_70_new.loc[index_70,'high_credit_amt']

index_30=raw_account_30_new['creditlimit'].index[raw_account_30_new['creditlimit'].apply(np.isnan)]
raw_account_30_new.loc[index_30,'creditlimit']=raw_account_30_new.loc[index_30,'high_credit_amt']

#Group by customer number in accounts dataset
#total_diff_lastpaymt_opened_dt
raw_account_70_new_lastpaymntdate_sum=raw_account_70_new.groupby(['customer_no'],as_index=False,sort=False)[['ind_acc_70','diff_lastpaymt_opened_dt','cur_balance_amt','creditlimit','cashlimit']].sum()
raw_account_30_new_lastpaymntdate_sum=raw_account_30_new.groupby(['customer_no'],as_index=False,sort=False)[['ind_acc_30','diff_lastpaymt_opened_dt','cur_balance_amt','creditlimit','cashlimit']].sum()
#mean
raw_account_70_new_lastpaymntdate_sum['mean_diff_lastpaymt_opened_dt']=raw_account_70_new_lastpaymntdate_sum['diff_lastpaymt_opened_dt']/raw_account_70_new_lastpaymntdate_sum['ind_acc_70']
raw_account_70_new_lastpaymntdate_sum['ratio_currbalance_creditlimit']=raw_account_70_new_lastpaymntdate_sum['cur_balance_amt']/raw_account_70_new_lastpaymntdate_sum['creditlimit']
raw_account_70_new_lastpaymntdate_sum['mean_curr_bal']=raw_account_70_new_lastpaymntdate_sum['cur_balance_amt']/raw_account_70_new_lastpaymntdate_sum['ind_acc_70']
raw_account_70_new_lastpaymntdate_sum['mean_credit_limit']=raw_account_70_new_lastpaymntdate_sum['creditlimit']/raw_account_70_new_lastpaymntdate_sum['ind_acc_70']
raw_account_70_new_lastpaymntdate_sum['mean_cashlimit']=raw_account_70_new_lastpaymntdate_sum['cashlimit']/raw_account_70_new_lastpaymntdate_sum['ind_acc_70']
raw_account_70_new_lastpaymntdate_sum['utilization_trend']=(raw_account_70_new_lastpaymntdate_sum['ratio_currbalance_creditlimit'])/(raw_account_70_new_lastpaymntdate_sum['mean_curr_bal']/(raw_account_70_new_lastpaymntdate_sum['mean_credit_limit']+raw_account_70_new_lastpaymntdate_sum['mean_cashlimit']))



raw_account_30_new_lastpaymntdate_sum['mean_diff_lastpaymt_opened_dt']=raw_account_30_new_lastpaymntdate_sum['diff_lastpaymt_opened_dt']/raw_account_30_new_lastpaymntdate_sum['ind_acc_30']
raw_account_30_new_lastpaymntdate_sum['ratio_currbalance_creditlimit']=raw_account_30_new_lastpaymntdate_sum['cur_balance_amt']/raw_account_30_new_lastpaymntdate_sum['creditlimit']
raw_account_30_new_lastpaymntdate_sum['mean_curr_bal']=raw_account_30_new_lastpaymntdate_sum['cur_balance_amt']/raw_account_30_new_lastpaymntdate_sum['ind_acc_30']
raw_account_30_new_lastpaymntdate_sum['mean_credit_limit']=raw_account_30_new_lastpaymntdate_sum['creditlimit']/raw_account_30_new_lastpaymntdate_sum['ind_acc_30']
raw_account_30_new_lastpaymntdate_sum['mean_cashlimit']=raw_account_30_new_lastpaymntdate_sum['cashlimit']/raw_account_30_new_lastpaymntdate_sum['ind_acc_30']
raw_account_30_new_lastpaymntdate_sum['utilization_trend']=(raw_account_30_new_lastpaymntdate_sum['ratio_currbalance_creditlimit'])/(raw_account_30_new_lastpaymntdate_sum['mean_curr_bal']/(raw_account_30_new_lastpaymntdate_sum['mean_credit_limit']+raw_account_30_new_lastpaymntdate_sum['mean_cashlimit']))

#Merge to demographic data
raw_data_70_new=pd.merge(raw_data_70_new,raw_account_70_new_lastpaymntdate_sum.loc[:,['customer_no','mean_diff_lastpaymt_opened_dt','ratio_currbalance_creditlimit','mean_curr_bal','mean_credit_limit','mean_cashlimit','utilization_trend']],how='left',on='customer_no')
raw_data_30_new=pd.merge(raw_data_30_new,raw_account_30_new_lastpaymntdate_sum.loc[:,['customer_no','mean_diff_lastpaymt_opened_dt','ratio_currbalance_creditlimit','mean_curr_bal','mean_credit_limit','mean_cashlimit','utilization_trend']],how='left',on='customer_no')


In [511]:
#Creating Secured and unsecured feature
raw_enquiry_70_new['sec_unsec_loan_ind']=[1 if i in [1,2,3,4,7,11,13,14,15,17,31,32,33,34,42,51,52,53,54,55,56,57,58,59,60] else 0 for i in raw_enquiry_70_new.enq_purpose]
raw_enquiry_30_new['sec_unsec_loan_ind']=[1 if i in [1,2,3,4,7,11,13,14,15,17,31,32,33,34,42,51,52,53,54,55,56,57,58,59,60] else 0 for i in raw_enquiry_30_new.enq_purpose]   

In [512]:
#enquiry_recency_365
raw_enquiry_70_new.enquiry_dt=pd.to_datetime(raw_enquiry_70_new.enquiry_dt)
raw_enquiry_70_new.dt_opened=pd.to_datetime(raw_enquiry_70_new.dt_opened)
raw_enquiry_30_new.enquiry_dt=pd.to_datetime(raw_enquiry_30_new.enquiry_dt)
raw_enquiry_30_new.dt_opened=pd.to_datetime(raw_enquiry_30_new.dt_opened)
raw_enquiry_70_new['enquiry_recency']=raw_enquiry_70_new.dt_opened-raw_enquiry_70_new.enquiry_dt
raw_enquiry_30_new['enquiry_recency']=raw_enquiry_30_new.dt_opened-raw_enquiry_30_new.enquiry_dt
raw_enquiry_70_new['enquiry_recency']=raw_enquiry_70_new.enquiry_recency.apply(lambda x: x / np.timedelta64(1,'D'))
raw_enquiry_30_new['enquiry_recency']=raw_enquiry_30_new.enquiry_recency.apply(lambda x: x / np.timedelta64(1,'D'))
raw_enquiry_70_new['count_enquiry_recency_365']=np.where(raw_enquiry_70_new.eval("enquiry_recency >0  and enquiry_recency <= 365"), 1, 0)
raw_enquiry_30_new['count_enquiry_recency_365']=np.where(raw_enquiry_30_new.eval("enquiry_recency >0  and enquiry_recency <= 365"), 1, 0)
raw_enquiry_70_new['count_enquiry_recency_90']=np.where(raw_enquiry_70_new.eval("enquiry_recency >0  and enquiry_recency <= 90"), 1, 0)
raw_enquiry_30_new['count_enquiry_recency_90']=np.where(raw_enquiry_30_new.eval("enquiry_recency >0  and enquiry_recency <= 90"), 1, 0)

#Group by customer number in accounts dataset
raw_enquiry_70_new_count_enquiry_recency_365=raw_enquiry_70_new.groupby(['customer_no'],as_index=False,sort=False)[['ind_enq_70','enquiry_recency','count_enquiry_recency_365','count_enquiry_recency_90','sec_unsec_loan_ind']].sum()
raw_enquiry_70_new_count_enquiry_recency_365['mean_enquiry_recency']=raw_enquiry_70_new_count_enquiry_recency_365['enquiry_recency']/raw_enquiry_70_new_count_enquiry_recency_365['ind_enq_70']
raw_enquiry_70_new_count_enquiry_recency_365['perc_unsecured_others']=raw_enquiry_70_new_count_enquiry_recency_365['sec_unsec_loan_ind']/raw_enquiry_70_new_count_enquiry_recency_365['ind_enq_70']


raw_enquiry_30_new_count_enquiry_recency_365=raw_enquiry_30_new.groupby(['customer_no'],as_index=False,sort=False)[['ind_enq_30','enquiry_recency','count_enquiry_recency_365','count_enquiry_recency_90','sec_unsec_loan_ind']].sum()
raw_enquiry_30_new_count_enquiry_recency_365['mean_enquiry_recency']=raw_enquiry_30_new_count_enquiry_recency_365['enquiry_recency']/raw_enquiry_30_new_count_enquiry_recency_365['ind_enq_30']
raw_enquiry_30_new_count_enquiry_recency_365['perc_unsecured_others']=raw_enquiry_30_new_count_enquiry_recency_365['sec_unsec_loan_ind']/raw_enquiry_30_new_count_enquiry_recency_365['ind_enq_30']


dummy_db=pd.DataFrame(raw_enquiry_70_new.groupby(['customer_no','enq_purpose'],as_index=False,sort=False)[['ind_enq_70']].count())
dummy_db=dummy_db.sort_values(['customer_no','ind_enq_70'],ascending=[1,0])
max_enq_purpose=pd.DataFrame(dummy_db.groupby('customer_no',as_index=False)[['enq_purpose']].first())
dummy_db_2=pd.DataFrame({'customer_no':max_enq_purpose['customer_no'],'max_enq_purpose':max_enq_purpose['enq_purpose']})
raw_data_70_new=pd.merge(raw_data_70_new,dummy_db_2,how='left',on='customer_no')

dummy_db=pd.DataFrame(raw_enquiry_30_new.groupby(['customer_no','enq_purpose'],as_index=False,sort=False)[['ind_enq_30']].count())
dummy_db=dummy_db.sort_values(['customer_no','ind_enq_30'],ascending=[1,0])
max_enq_purpose=pd.DataFrame(dummy_db.groupby('customer_no',as_index=False)[['enq_purpose']].first())
dummy_db_2=pd.DataFrame({'customer_no':max_enq_purpose['customer_no'],'max_enq_purpose':max_enq_purpose['enq_purpose']})

raw_data_30_new=pd.merge(raw_data_30_new,dummy_db_2,how='left',on='customer_no')





#Merge to demographic data

raw_data_70_new=pd.merge(raw_data_70_new,raw_enquiry_70_new_count_enquiry_recency_365.loc[:,['customer_no','ind_enq_70','count_enquiry_recency_365','mean_enquiry_recency','count_enquiry_recency_90','perc_unsecured_others']],how='left',on='customer_no')
raw_data_30_new=pd.merge(raw_data_30_new,raw_enquiry_30_new_count_enquiry_recency_365.loc[:,['customer_no','ind_enq_30','count_enquiry_recency_365','mean_enquiry_recency','count_enquiry_recency_90','perc_unsecured_others']],how='left',on='customer_no')


In [513]:
raw_data_70_new.head(10)

,dt_opened,customer_no,entry_time,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,mean_curr_bal,mean_credit_limit,mean_cashlimit,utilization_trend,max_enq_purpose,ind_enq_70,count_enquiry_recency_365,mean_enquiry_recency,count_enquiry_recency_90,perc_unsecured_others
0,18-Apr-15,1,13-Apr-15,Insignia,13-Apr-15,650.0,2.0,Card Setup,14.0,500000.0,...,2.619365e+05,4.530004e+05,9333.333333,1.020603,10.0,18,1,1443.500000,0,0.055556
1,21-Apr-15,2,21-Apr-15,Insignia,21-Apr-15,760.0,1.0,Card Setup,14.0,1200000.0,...,1.537700e+04,5.587975e+05,0.500000,1.000001,51.0,68,19,812.485294,2,0.544118
2,22-Apr-15,3,21-Apr-15,Insignia,21-Apr-15,774.0,1.0,Card Setup,14.0,700000.0,...,1.786400e+04,1.215230e+05,0.000000,1.000000,1.0,1,0,438.000000,0,1.000000
3,25-Apr-15,4,15-Apr-15,Insignia,20-Apr-15,770.0,1.0,Card Setup,14.0,500000.0,...,1.085629e+05,7.556722e+05,21235.294118,1.028101,51.0,34,0,1920.823529,0,0.529412
4,6-May-15,5,30-Apr-15,Insignia,NaN,NaN,3.0,Card Setup,14.0,500000.0,...,1.139000e+03,1.831072e+06,0.000000,1.000000,10.0,2,0,1497.500000,0,0.000000
5,12-May-15,6,6-May-15,Insignia,6-May-15,779.0,1.0,Card Setup,14.0,1000000.0,...,6.609884e+05,2.432064e+06,15000.027778,1.006168,2.0,27,3,1725.370370,0,0.481481
6,22-May-15,7,6-May-15,Insignia,6-May-15,762.0,1.0,Card Setup,14.0,100000.0,...,7.307714e+03,4.529129e+04,16557.142857,1.365570,10.0,1,0,3258.000000,0,0.000000
7,22-May-15,8,15-May-15,Insignia,15-May-15,799.0,1.0,Card Setup,14.0,357000.0,...,9.352719e+05,1.374843e+06,16600.000000,1.012074,1.0,17,6,1158.588235,0,0.588235
8,28-May-15,9,15-May-15,Insignia,15-May-15,762.0,1.0,Card Setup,14.0,550000.0,...,2.354439e+06,3.355187e+06,16833.500000,1.005017,1.0,19,6,891.473684,0,0.631579
9,30-May-15,10,27-May-15,Insignia,27-May-15,784.0,1.0,Card Setup,14.0,1017000.0,...,2.064325e+04,6.801118e+05,20250.000000,1.029775,1.0,21,7,897.571429,1,0.428571


In [514]:
raw_data_30_new.head(10)

,dt_opened,customer_no,entry_time,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,mean_curr_bal,mean_credit_limit,mean_cashlimit,utilization_trend,max_enq_purpose,ind_enq_30,count_enquiry_recency_365,mean_enquiry_recency,count_enquiry_recency_90,perc_unsecured_others
0,21-Apr-15,1,17-Apr-15,Insignia,17-Apr-15,726.0,1.0,Card Setup,14.0,1200000.0,...,5.756704e+06,9.240433e+06,17531.250000,1.001897,2.0,33,3,1343.575758,0,0.696970
1,12-May-15,2,6-May-15,Insignia,6-May-15,844.0,1.0,Card Setup,14.0,300000.0,...,-7.600000e+01,2.901700e+04,0.000000,1.000000,10.0,2,0,1370.500000,0,0.000000
2,21-May-15,3,20-May-15,Insignia,20-May-15,723.0,3.0,Card Setup,14.0,1500000.0,...,1.757027e+06,3.740570e+06,47127.272727,1.012599,2.0,17,1,1914.705882,1,0.470588
3,22-May-15,4,22-May-15,Insignia,22-May-15,778.0,1.0,Card Setup,14.0,2500000.0,...,4.563415e+05,1.616093e+06,69515.240000,1.043014,5.0,50,2,2016.400000,0,0.380000
4,22-May-15,5,12-May-15,Insignia,13-May-15,810.0,1.0,Card Setup,14.0,500000.0,...,7.706667e+03,1.516983e+05,38333.333333,1.252694,51.0,5,2,633.800000,0,0.600000
5,27-May-15,6,21-May-15,Insignia,21-May-15,601.0,2.0,Card Setup,14.0,500000.0,...,5.569923e+06,9.153389e+06,21866.666667,1.002389,10.0,13,2,1128.307692,0,0.384615
6,10-Jun-15,7,6-Jun-15,Insignia,6-Jun-15,712.0,1.0,Card Setup,14.0,1000000.0,...,6.300850e+04,6.125000e+05,0.000000,1.000000,10.0,9,3,453.666667,0,0.111111
7,27-Jun-15,8,24-Jun-15,Insignia,24-Jun-15,620.0,2.0,Card Setup,14.0,1000000.0,...,3.502761e+04,1.738661e+05,29768.500000,1.171215,10.0,11,1,1905.454545,1,0.000000
8,27-Jun-15,9,24-Jun-15,Insignia,24-Jun-15,772.0,1.0,Card Setup,14.0,500000.0,...,4.111111e+02,1.628100e+04,2222.222222,1.136492,10.0,17,2,1894.588235,0,0.235294
9,4-Jun-15,10,7-May-15,Platinum Maxima,7-May-15,702.0,3.0,Card Setup,14.0,100000.0,...,7.461970e+04,1.606233e+05,4135.185185,1.025745,5.0,100,8,1779.680000,0,0.070000


# Fill Factor Analysis
- Fill factor analysis is done on dataset to understand the percentage of missing values
- This we will do only on data_70 as it is training dataset. data_30 will be considered as test dataset.Column we take in train will be finalized in test

In [515]:
def fill_percentage(dataset,threshold=0.5):
    '''function to calculate fill percentage and 
    return column names with fill percentage above threshold'''
    column_names=list(dataset)
    number_of_rows=dataset.shape[1]
    count=0
    column_list=[]
    for column in column_names:
        number_of_non_missing_values=number_of_rows-sum(dataset[column].isnull())
        perc_non_missing=number_of_non_missing_values/number_of_rows
        if perc_non_missing>=threshold:
            print("Column"+"-"+column+":"+str(perc_non_missing))
            count=count+1
            column_list.append(column)
    print("Total number of columns above threshold"+":"+str(count))
    return column_list

In [516]:
column=fill_percentage(dataset=raw_data_70_new,threshold=0.5)

Column-dt_opened:1.0
Column-customer_no:1.0
Column-entry_time:0.8529411764705882
Column-feature_1:0.8529411764705882
Column-feature_4:0.8529411764705882
Column-feature_5:0.8529411764705882
Column-feature_6:0.8529411764705882
Column-feature_7:0.8529411764705882
Column-feature_11:0.8529411764705882
Column-feature_12:0.8529411764705882
Column-feature_15:0.7745098039215687
Column-feature_16:0.7352941176470589
Column-feature_19:0.8529411764705882
Column-feature_20:1.0
Column-feature_21:0.8529411764705882
Column-feature_22:1.0
Column-feature_23:0.8529411764705882
Column-feature_25:0.8529411764705882
Column-feature_26:0.8529411764705882
Column-feature_28:0.8529411764705882
Column-feature_29:0.8529411764705882
Column-feature_30:0.8529411764705882
Column-feature_31:0.8529411764705882
Column-feature_32:0.8529411764705882
Column-feature_33:0.8529411764705882
Column-feature_34:0.8529411764705882
Column-feature_35:0.8529411764705882
Column-feature_39:0.8529411764705882
Column-feature_40:0.852941176

In [517]:
# Column created with date difference with 2015 and some columns to retain pattern if any
raw_data_70_new['diff_feature_30']=2015-raw_data_70_new['feature_30']
raw_data_70_new['diff_feature_39']=2015-raw_data_70_new['feature_39']
raw_data_30_new['diff_feature_30']=2015-raw_data_30_new['feature_30']
raw_data_30_new['diff_feature_39']=2015-raw_data_30_new['feature_39']
column.extend(['diff_feature_30','diff_feature_39'])

In [518]:
#Removing column with dates , phone number and other column with redundant values
redun_col_list=['dt_opened','entry_time','feature_1','feature_2','feature_5','feature_6','feature_20','feature_21','feature_22','feature_24','feature_28','feature_29','feature_30','feature_39','feature_43','feature_44','feature_47','feature_54','feature_63','feature_66','feature_70','feature_75','feature_77','feature_58','feature_59','feature_55','feature_62','feature_79','feature_78','feature_76','feature_19','feature_69','feature_67','feature_68','feature_25']
for i in redun_col_list:
        if i in column:
            column.remove(i)
print(column)
print(len(column))

['customer_no', 'feature_4', 'feature_7', 'feature_11', 'feature_12', 'feature_15', 'feature_16', 'feature_23', 'feature_26', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_40', 'feature_41', 'feature_42', 'feature_50', 'feature_52', 'feature_56', 'feature_60', 'feature_64', 'feature_65', 'feature_71', 'feature_72', 'Bad_label', 'ind_acc_70', 'payment_history_mean_length', 'months_last_30_plus', 'mean_mon_30p', 'current_age_max', 'mean_diff_lastpaymt_opened_dt', 'ratio_currbalance_creditlimit', 'mean_curr_bal', 'mean_credit_limit', 'mean_cashlimit', 'ind_enq_70', 'count_enquiry_recency_365', 'count_enquiry_recency_90', 'perc_unsecured_others', 'diff_feature_30', 'diff_feature_39']
42


# Information value Analysis
- Information value is calculated to check column with very low values

In [519]:
max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [520]:
final_iv,IV=data_vars(raw_data_70_new.loc[:,column],raw_data_70_new.Bad_label)

/Users/alokanand/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:84: RuntimeWarning: divide by zero encountered in log
/Users/alokanand/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:85: RuntimeWarning: divide by zero encountered in log
/Users/alokanand/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: divide by zero encountered in log
/Users/alokanand/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in log


In [521]:
IV.sort_values(['IV'],ascending=False)

,VAR_NAME,IV
9,feature_16,0.229642
27,feature_7,0.163966
8,feature_15,0.158032
22,feature_52,0.102968
21,feature_50,0.092260
35,mean_diff_lastpaymt_opened_dt,0.085221
32,mean_cashlimit,0.081758
0,count_enquiry_recency_365,0.074623
38,payment_history_mean_length,0.074106
1,count_enquiry_recency_90,0.063249


In [522]:
#Train data and test data
train=raw_data_70_new.loc[:,column]
test=raw_data_30_new.loc[:,column]

In [523]:
#Imputing missing values,all missing values with -999
train.fillna(value=-999, inplace=True)
test.fillna(value=-999, inplace=True)


In [524]:
#Train data info
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23896 entries, 0 to 23895
Data columns (total 42 columns):
customer_no                      23896 non-null int64
feature_4                        23896 non-null float64
feature_7                        23896 non-null float64
feature_11                       23896 non-null object
feature_12                       23896 non-null object
feature_15                       23896 non-null object
feature_16                       23896 non-null object
feature_23                       23896 non-null object
feature_26                       23896 non-null float64
feature_31                       23896 non-null float64
feature_32                       23896 non-null object
feature_33                       23896 non-null object
feature_34                       23896 non-null float64
feature_35                       23896 non-null float64
feature_40                       23896 non-null float64
feature_41                       23896 non-null float64
feat

In [525]:
# Test data info
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10240 entries, 0 to 10239
Data columns (total 42 columns):
customer_no                      10240 non-null int64
feature_4                        10240 non-null float64
feature_7                        10240 non-null float64
feature_11                       10240 non-null object
feature_12                       10240 non-null object
feature_15                       10240 non-null object
feature_16                       10240 non-null object
feature_23                       10240 non-null object
feature_26                       10240 non-null float64
feature_31                       10240 non-null float64
feature_32                       10240 non-null object
feature_33                       10240 non-null object
feature_34                       10240 non-null float64
feature_35                       10240 non-null float64
feature_40                       10240 non-null float64
feature_41                       10240 non-null float64
feat

In [526]:
#Before sampling and training dataset we have to change object values to numerical
def object_numeric(dataset):
    object_col=list(dataset.select_dtypes(include=[object]))
    object_dict={-999:-999}
    for i in object_col:
        object_value_list=list(dataset[i].unique())
        if -999 in object_value_list:
            object_value_list.remove(-999)
        for j in range(len(object_value_list)):
            object_dict[object_value_list[j]]=j
        dataset[i] = [object_dict[item] for item in dataset[i]]
    return dataset
train=object_numeric(dataset=train)
test=object_numeric(dataset=test)

# Sampling data to balance from Imbalance in train 
- For training data we will create 3 sampled dataset with 6000 0s and 1004 1s  to balance the label and emsemble in the end to enhance the result

In [527]:
#Storing Postive and neg indices
index_pos=list(train['Bad_label'].index[train['Bad_label']==1])
index_neg=list(train['Bad_label'].index[train['Bad_label']!=1])

In [549]:
random.seed(100)

In [528]:
#Taking first sample from full index_neg without replacement 
index_neg_fin=random.sample(set(index_neg), 6000)
for i in index_neg_fin:
    if i in index_neg:
        index_neg.remove(i)

In [529]:
#Taking second sample from full index_neg without replacement 
index_neg_sec=random.sample(set(index_neg), 6000)
for i in index_neg_sec:
    if i in index_neg:
        index_neg.remove(i)

In [530]:
#Taking third sample from full index_neg without replacement 
index_neg_thr=random.sample(set(index_neg), 6000)
for i in index_neg_thr:
    if i in index_neg:
        index_neg.remove(i)

In [531]:
# Merging data and creating three training datasets
train_fin=pd.concat([train.loc[index_neg_fin,:],train.loc[index_pos,:]],axis=0)
train_sec=pd.concat([train.loc[index_neg_sec,:],train.loc[index_pos,:]],axis=0)
train_thr=pd.concat([train.loc[index_neg_thr,:],train.loc[index_pos,:]],axis=0)
print(train_fin.shape)
print(train_sec.shape)
print(train_thr.shape)

(7004, 42)
(7004, 42)
(7004, 42)


In [532]:
# Label Percentage .same for all
train_fin.Bad_label.value_counts(normalize=True)

0    0.856653
1    0.143347
Name: Bad_label, dtype: float64

# Training data using Xgboost

In [533]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

# Training First Dataset

In [534]:
def modelfit (alg, dtrain , dtest, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Bad_label'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    #predit Test set
    dtest_predictions = alg.predict(dtest[predictors])
    dtest_predprob = alg.predict_proba(dtest[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy(Train) : %.4g" % metrics.accuracy_score(dtrain['Bad_label'].values, dtrain_predictions))
    print ("Accuracy(Test) : %.4g" % metrics.accuracy_score(dtest['Bad_label'].values, dtest_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Bad_label'], dtrain_predprob))
    print ("AUC Score (Test): %f" % metrics.roc_auc_score(dtest['Bad_label'], dtest_predprob))

In [535]:
target = 'Bad_label'
IDcol ='customer_no'

In [536]:
#Choose all predictors except target & IDcols
predictors = list(train_fin.columns)
predictors.remove(target)
predictors.remove(IDcol)
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train_fin,test, predictors)


Model Report
Accuracy(Train) : 0.8625
Accuracy(Test) : 0.9539
AUC Score (Train): 0.871417
AUC Score (Test): 0.649483


/Users/alokanand/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/alokanand/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


-Model Overfitting

In [537]:
#Tune max_depth and min_child_weight
param_test1 = {
 'max_depth':list(range(3,10,2)),
 'min_child_weight':list(range(1,6,2))
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=200, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train_fin[predictors],train_fin[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.43602, std: 0.15134, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.43891, std: 0.14749, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.42986, std: 0.14437, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.42576, std: 0.14039, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.43298, std: 0.14013, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.42728, std: 0.13471, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.42733, std: 0.14001, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.43032, std: 0.13393, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.42945, std: 0.12811, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.42857, std: 0.12745, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.42771, std: 0.12950, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.42807, std: 0.11882, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_child_weight': 3

In [538]:
param_test2 = {
 'max_depth':[2,3,4,5],
 'min_child_weight':[2,3,4,5]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=200, max_depth=3,
 min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train_fin[predictors],train_fin[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: 0.45098, std: 0.15196, params: {'max_depth': 2, 'min_child_weight': 2},
  mean: 0.45209, std: 0.15139, params: {'max_depth': 2, 'min_child_weight': 3},
  mean: 0.45166, std: 0.15424, params: {'max_depth': 2, 'min_child_weight': 4},
  mean: 0.45176, std: 0.15141, params: {'max_depth': 2, 'min_child_weight': 5},
  mean: 0.43533, std: 0.14804, params: {'max_depth': 3, 'min_child_weight': 2},
  mean: 0.43891, std: 0.14749, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.42926, std: 0.14591, params: {'max_depth': 3, 'min_child_weight': 4},
  mean: 0.42986, std: 0.14437, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.42377, std: 0.13735, params: {'max_depth': 4, 'min_child_weight': 2},
  mean: 0.42778, std: 0.14555, params: {'max_depth': 4, 'min_child_weight': 3},
  mean: 0.42118, std: 0.13658, params: {'max_depth': 4, 'min_child_weight': 4},
  mean: 0.42345, std: 0.13860, params: {'max_depth': 4, 'min_child_weight': 5},
  mean: 0.42291, std: 0.13471, params: {

In [539]:
#Tune gamma
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=200, max_depth=2,
 min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(train_fin[predictors],train_fin[target])
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: 0.45209, std: 0.15139, params: {'gamma': 0.0},
  mean: 0.45263, std: 0.15178, params: {'gamma': 0.1},
  mean: 0.45256, std: 0.15176, params: {'gamma': 0.2},
  mean: 0.45373, std: 0.15313, params: {'gamma': 0.3},
  mean: 0.45371, std: 0.15313, params: {'gamma': 0.4}],
 {'gamma': 0.3},
 0.4537257918739635)

In [540]:
#Tune subsample and colsample_bytree
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=200, max_depth=2,
 min_child_weight=3, gamma=0.3, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(train_fin[predictors],train_fin[target])
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: 0.45302, std: 0.14821, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.45315, std: 0.15449, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.45314, std: 0.15569, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.44918, std: 0.15754, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.45336, std: 0.15053, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.45624, std: 0.15519, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.45948, std: 0.15546, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.45261, std: 0.15809, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.45414, std: 0.14782, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.46031, std: 0.15336, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.45373, std: 0.15313, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.45103, std: 0.15607, params: {'colsample_bytree': 0.8, 'subsample'

In [541]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=200,
 max_depth=2,
 min_child_weight=3,
 gamma=0.3,
 subsample=0.7,
 colsample_bytree=0.8,
 reg_alpha=0.001,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1,train_fin,test, predictors)


Model Report
Accuracy(Train) : 0.8579
Accuracy(Test) : 0.949
AUC Score (Train): 0.764838
AUC Score (Test): 0.652438


/Users/alokanand/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/alokanand/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [563]:
#feature Importance on first dataset
feature_imp1=pd.DataFrame({'Columns':predictors,'Importance':xgb1.feature_importances_})
feature_imp1.sort_values(['Importance'],ascending=False)
feature_imp1.to_csv('feature_imp1.csv')

# Training Second Dataset

In [543]:
#Choose all predictors except target & IDcols
predictors = list(train_sec.columns)
predictors.remove(target)
predictors.remove(IDcol)
xgb2 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=200,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb2, train_sec,test, predictors)


Model Report
Accuracy(Train) : 0.8659
Accuracy(Test) : 0.9505
AUC Score (Train): 0.885801
AUC Score (Test): 0.641415


/Users/alokanand/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/alokanand/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [544]:
#Tune max_depth and min_child_weight
param_test1 = {
 'max_depth':list(range(3,10,2)),
 'min_child_weight':list(range(1,6,2))
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=200, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train_sec[predictors],train_sec[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.42487, std: 0.14786, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.42236, std: 0.14837, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.42523, std: 0.14626, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.41515, std: 0.14510, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.42041, std: 0.14302, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.42218, std: 0.14412, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.42464, std: 0.14431, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.41708, std: 0.13173, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.41575, std: 0.13447, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.42670, std: 0.14016, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.41585, std: 0.12568, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.42401, std: 0.13293, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 9, 'min_child_weight': 1

In [545]:
param_test2 = {
 'max_depth':[2,3,4,5,6,7,8,9,10],
 'min_child_weight':[1,2,3,4,5]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=200, max_depth=9,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train_sec[predictors],train_sec[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: 0.44944, std: 0.15736, params: {'max_depth': 2, 'min_child_weight': 1},
  mean: 0.44662, std: 0.15575, params: {'max_depth': 2, 'min_child_weight': 2},
  mean: 0.45159, std: 0.15904, params: {'max_depth': 2, 'min_child_weight': 3},
  mean: 0.45292, std: 0.15949, params: {'max_depth': 2, 'min_child_weight': 4},
  mean: 0.44924, std: 0.15701, params: {'max_depth': 2, 'min_child_weight': 5},
  mean: 0.42487, std: 0.14786, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.42597, std: 0.14775, params: {'max_depth': 3, 'min_child_weight': 2},
  mean: 0.42236, std: 0.14837, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.42667, std: 0.14936, params: {'max_depth': 3, 'min_child_weight': 4},
  mean: 0.42523, std: 0.14626, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.42464, std: 0.14932, params: {'max_depth': 4, 'min_child_weight': 1},
  mean: 0.42274, std: 0.14622, params: {'max_depth': 4, 'min_child_weight': 2},
  mean: 0.42201, std: 0.14705, params: {

In [546]:
#Tune gamma
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=200, max_depth=2,
 min_child_weight=4, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(train_sec[predictors],train_sec[target])
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: 0.45292, std: 0.15949, params: {'gamma': 0.0},
  mean: 0.45230, std: 0.15900, params: {'gamma': 0.1},
  mean: 0.45089, std: 0.15908, params: {'gamma': 0.2},
  mean: 0.45089, std: 0.15908, params: {'gamma': 0.3},
  mean: 0.45094, std: 0.15914, params: {'gamma': 0.4}],
 {'gamma': 0.0},
 0.4529197989220564)

In [547]:
#Tune subsample and colsample_bytree
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=200, max_depth=2,
 min_child_weight=4, gamma=0.0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(train_sec[predictors],train_sec[target])
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: 0.44934, std: 0.15978, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.44565, std: 0.15735, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.44652, std: 0.15713, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.44767, std: 0.16324, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.45578, std: 0.15846, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.44632, std: 0.15686, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.44498, std: 0.15757, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.44773, std: 0.15864, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.45276, std: 0.15551, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.44514, std: 0.15568, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.45292, std: 0.15949, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.44497, std: 0.15818, params: {'colsample_bytree': 0.8, 'subsample'

In [480]:
xgb2 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=200,
 max_depth=2,
 min_child_weight=4,
 gamma=0,
 subsample=0.6,
 colsample_bytree=0.7,
 reg_alpha=0.001,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb2, train_sec,test, predictors)


Model Report
Accuracy(Train) : 0.8578
Accuracy(Test) : 0.9534
AUC Score (Train): 0.744439
AUC Score (Test): 0.659351


/Users/alokanand/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/alokanand/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [562]:
#feature Importance on second dataset
feature_imp2=pd.DataFrame({'Columns':predictors,'Importance':xgb2.feature_importances_})
feature_imp2.sort_values(['Importance'],ascending=False)
feature_imp2.to_csv('feature_imp2.csv')

# Training Third Dataset

In [551]:
#Choose all predictors except target & IDcols
predictors = list(train_thr.columns)
predictors.remove(target)
predictors.remove(IDcol)
xgb3 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=200,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb3, train_thr,test, predictors)


Model Report
Accuracy(Train) : 0.8652
Accuracy(Test) : 0.952
AUC Score (Train): 0.864196
AUC Score (Test): 0.655401


/Users/alokanand/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/alokanand/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [552]:
#Tune max_depth and min_child_weight
param_test1 = {
 'max_depth':list(range(3,10,2)),
 'min_child_weight':list(range(1,6,2))
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=200, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train_thr[predictors],train_thr[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.43357, std: 0.15208, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.42603, std: 0.14767, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.42315, std: 0.14926, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.41344, std: 0.14564, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.40862, std: 0.14620, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.41185, std: 0.14140, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.40698, std: 0.13464, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.41273, std: 0.13292, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.40828, std: 0.12608, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.42419, std: 0.13563, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.40845, std: 0.12851, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.41322, std: 0.12681, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_child_weight': 1

In [553]:
param_test2 = {
 'max_depth':[2,3,4,5],
 'min_child_weight':[1,2,3,4,5]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=200, max_depth=3,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train_thr[predictors],train_thr[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: 0.44548, std: 0.15861, params: {'max_depth': 2, 'min_child_weight': 1},
  mean: 0.44617, std: 0.15739, params: {'max_depth': 2, 'min_child_weight': 2},
  mean: 0.44404, std: 0.15796, params: {'max_depth': 2, 'min_child_weight': 3},
  mean: 0.44489, std: 0.15497, params: {'max_depth': 2, 'min_child_weight': 4},
  mean: 0.43895, std: 0.15712, params: {'max_depth': 2, 'min_child_weight': 5},
  mean: 0.43357, std: 0.15208, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.42785, std: 0.14990, params: {'max_depth': 3, 'min_child_weight': 2},
  mean: 0.42603, std: 0.14767, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.42527, std: 0.15048, params: {'max_depth': 3, 'min_child_weight': 4},
  mean: 0.42315, std: 0.14926, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.41652, std: 0.14707, params: {'max_depth': 4, 'min_child_weight': 1},
  mean: 0.42049, std: 0.14856, params: {'max_depth': 4, 'min_child_weight': 2},
  mean: 0.41845, std: 0.14658, params: {

In [554]:
#Tune gamma
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=200, max_depth=2,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(train_thr[predictors],train_thr[target])
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: 0.44617, std: 0.15739, params: {'gamma': 0.0},
  mean: 0.44669, std: 0.15781, params: {'gamma': 0.1},
  mean: 0.44669, std: 0.15781, params: {'gamma': 0.2},
  mean: 0.44675, std: 0.15779, params: {'gamma': 0.3},
  mean: 0.44675, std: 0.15779, params: {'gamma': 0.4}],
 {'gamma': 0.4},
 0.4467466873963516)

In [555]:
#Tune subsample and colsample_bytree
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=200, max_depth=2,
 min_child_weight=3, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(train_thr[predictors],train_thr[target])
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: 0.45498, std: 0.15109, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.45301, std: 0.15359, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.44653, std: 0.15870, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.44834, std: 0.16108, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.45800, std: 0.15338, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.45357, std: 0.15442, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.44527, std: 0.15986, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.44258, std: 0.15538, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.45465, std: 0.15405, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.45123, std: 0.15507, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.44533, std: 0.15820, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.44597, std: 0.16071, params: {'colsample_bytree': 0.8, 'subsample'

In [556]:
xgb3 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=200,
 max_depth=2,
 min_child_weight=3,
 gamma=0.4,
 subsample=0.6,
 colsample_bytree=0.7,
 reg_alpha=0.001,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb3, train_thr,test, predictors)


Model Report
Accuracy(Train) : 0.8572
Accuracy(Test) : 0.9528
AUC Score (Train): 0.737303
AUC Score (Test): 0.658727


/Users/alokanand/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/alokanand/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [560]:
#feature Importance on third dataset
feature_imp3=pd.DataFrame({'Columns':predictors,'Importance':xgb3.feature_importances_})
feature_imp3.sort_values(['Importance'],ascending=False)
feature_imp3.to_csv('feature_imp3.csv')

# Ensemble of three models on Test dataset


In [558]:
test_result=pd.DataFrame({'customer_id':test.customer_no,'Bad_label':test.Bad_label,'pred1':xgb1.predict_proba(test[predictors])[:,1],'pred2':xgb2.predict_proba(test[predictors])[:,1],'pred3':xgb3.predict_proba(test[predictors])[:,1]})
test_result['pred_avg']=(test_result.pred1+test_result.pred2+test_result.pred3)/3
print('Final Auc:'+str(metrics.roc_auc_score(test_result['Bad_label'], test_result['pred_avg'])))
print('Gini:'+str(2*(metrics.roc_auc_score(test_result['Bad_label'], test_result['pred_avg']))-1))

Final Auc:0.6571725642599032
Gini:0.31434512851980645


In [559]:
test_result.to_csv('test_result.csv')